In [3]:
%pip install supabase

import requests
import pandas as pd

from io import StringIO

import supabase


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
base_url = "http://127.0.0.1:8000/"
market_data_endpoint = base_url + "market-data/"

In [48]:
resp = requests.get(market_data_endpoint + "historical?tickers=TSLA,AMZN&period=1d&interval=60m").json()

res = {}
for ticker, df_json in resp.items():
    res[ticker] = pd.read_json(
        StringIO(df_json),
        orient='index',
        convert_dates=True
    )

res

{'TSLA':                                 Close        High         Low        Open  \
 2025-08-15 13:30:00+00:00  332.899994  339.299988  331.850006  337.654999   
 2025-08-15 14:30:00+00:00  332.905792  333.799896  331.880005  332.885010   
 2025-08-15 15:30:00+00:00  327.940002  333.554993  327.019989  332.875000   
 2025-08-15 16:30:00+00:00  330.899994  331.429596  327.934998  327.934998   
 2025-08-15 17:30:00+00:00  329.829987  331.699890  329.339996  330.880096   
 2025-08-15 18:30:00+00:00  329.059998  330.119995  328.000000  329.832794   
 2025-08-15 19:30:00+00:00  330.600006  330.670013  328.570007  329.089996   
 
                              Volume  
 2025-08-15 13:30:00+00:00  18871909  
 2025-08-15 14:30:00+00:00   8164869  
 2025-08-15 15:30:00+00:00  12912763  
 2025-08-15 16:30:00+00:00   9426179  
 2025-08-15 17:30:00+00:00   6017263  
 2025-08-15 18:30:00+00:00   7713775  
 2025-08-15 19:30:00+00:00   6091565  ,
 'AMZN':                                 Close       

In [49]:
resp = requests.get(market_data_endpoint + "prices?tickers=TSLA,AMZN").json()
resp

{'TSLA': {'Datetime': '2025-08-15T15:59:00-04:00',
  'Open': 330.0799865722656,
  'High': 330.6700134277344,
  'Low': 329.9949951171875,
  'Close': 330.6000061035156,
  'Volume': 916453,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 'AMZN': {'Datetime': '2025-08-15T15:59:00-04:00',
  'Open': 230.97000122070312,
  'High': 231.1199951171875,
  'Low': 230.92999267578125,
  'Close': 231.05999755859375,
  'Volume': 790218,
  'Dividends': 0.0,
  'Stock Splits': 0.0}}

In [8]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# test_portfolios.py
import requests
import os
import json


# --------------------
# CONFIG
# --------------------
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")  # or anon key if email/password login
BASE_URL = "http://127.0.0.1:8000/api/1.0"

EMAIL = "test@example.com"
PASSWORD = "password123"

# --------------------
# 1. Get Supabase Access Token
# --------------------
auth_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if auth_res.status_code != 200:
    print("❌ Failed to log in:", auth_res.status_code, auth_res.text)
    exit(1)

access_token = auth_res.json()["access_token"]
print(f"✅ Logged in as {EMAIL}")
print(f"Access Token: {access_token[:20]}...")

# --------------------
# 2. Call /portfolios
# --------------------
headers = {
    "Authorization": f"Bearer {access_token}"
}

portfolios_res = requests.get(f"{BASE_URL}/portfolios", headers=headers)

print("\nStatus:", portfolios_res.status_code)
try:
    print(json.dumps(portfolios_res.json(), indent=4))
except Exception:
    print(portfolios_res.text)


✅ Logged in as test@example.com
Access Token: eyJhbGciOiJIUzI1NiIs...

Status: 404
{
    "detail": "Not Found"
}


In [ ]:
import requests
import os
import json

# ======== CONFIG ========
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")  # or anon key if using public route
API_BASE_URL = "http://127.0.0.1:8000/api/1.0"

EMAIL = "test@example.com"
PASSWORD = "password123"

# ======== STEP 1: LOGIN TO GET TOKEN ========
login_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if login_res.status_code != 200:
    print("❌ Failed to log in:", login_res.text)
    exit()

token = login_res.json()["access_token"]
print("✅ Got token:", token[:20] + "...")

# ======== STEP 2: CREATE PORTFOLIO ========

create_res = requests.post(
    f"{API_BASE_URL}/portfolios",
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    },
    json={
        "name": "Create Test Portfolio",
        "initial_investment": 10000.00,
    }
)
print("\n📌 Response Status:", create_res.status_code)
print("📌 Response Body:\n", json.dumps(create_res.json(), indent=4))


✅ Got token: eyJhbGciOiJIUzI1NiIs...

📌 Response Status: 200
📌 Response Body:
 {
    "id": "6e9fcc98-06ec-4f06-a81f-f31f99de616e",
    "created_at": "2025-08-15T22:42:45.581734+00:00",
    "name": "Create Test Portfolio",
    "last_updated": "2025-08-15T22:42:45.581734+00:00",
    "initial_investment": 10000.0,
    "user_id": "d62c7b0a-4af1-4520-9f35-ec7825d8c227"
}


In [ ]:
import requests
import os
import json

# --------------------
# CONFIG
# --------------------
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
BASE_URL = "http://127.0.0.1:8000/api/1.0"

EMAIL = "test@example.com"
PASSWORD = "password123"

# Portfolio UUID you want to delete
PORTFOLIO_ID = "57867417-1da2-4bf9-8129-f707898f34c2"

# --------------------
# 1. Get Supabase Access Token
# --------------------
auth_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if auth_res.status_code != 200:
    print("❌ Failed to log in:", auth_res.status_code, auth_res.text)
    exit(1)

access_token = auth_res.json()["access_token"]
print(f"✅ Logged in as {EMAIL}")
print(f"Access Token: {access_token[:20]}...")

# --------------------
# 2. Delete Portfolio
# --------------------
headers = {
    "Authorization": f"Bearer {access_token}"
}

delete_res = requests.delete(f"{BASE_URL}/portfolios/{PORTFOLIO_ID}", headers=headers)

print("\nStatus:", delete_res.status_code)
try:
    print(json.dumps(delete_res.json(), indent=4))
except Exception:
    print(delete_res.text)


✅ Logged in as test@example.com
Access Token: eyJhbGciOiJIUzI1NiIs...

Status: 500
Internal Server Error


In [ ]:
import requests
import os
import json

# --------------------
# CONFIG
# --------------------
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
BASE_URL = "http://127.0.0.1:8000/api/1.0"

EMAIL = "test@example.com"
PASSWORD = "password123"

# Portfolio UUID you want to delete
PORTFOLIO_ID = "591dd7f5-fff1-4041-ae74-3507688da719"

# --------------------
# 1. Get Supabase Access Token
# --------------------
auth_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if auth_res.status_code != 200:
    print("❌ Failed to log in:", auth_res.status_code, auth_res.text)
    exit(1)

access_token = auth_res.json()["access_token"]
print(f"✅ Logged in as {EMAIL}")
print(f"Access Token: {access_token[:20]}...")

# --------------------
# 2. Delete Portfolio
# --------------------
headers = {
    "Authorization": f"Bearer {access_token}"
}

delete_res = requests.get(f"{BASE_URL}/portfolios/{PORTFOLIO_ID}/positions", headers=headers)

print("\nStatus:", delete_res.status_code)
try:
    print(json.dumps(delete_res.json(), indent=4))
except Exception:
    print(delete_res.text)


✅ Logged in as test@example.com
Access Token: eyJhbGciOiJIUzI1NiIs...

Status: 200
{
    "status": "success",
    "positions": [
        {
            "portfolio_id": "591dd7f5-fff1-4041-ae74-3507688da719",
            "ticker": "AMZN",
            "quantity": 10,
            "created_at": "2025-08-15T23:14:39.057363+00:00",
            "updated_at": "2025-08-15T23:17:41.117032+00:00"
        }
    ]
}


In [ ]:
import requests
import os
import json

# --------------------
# CONFIG
# --------------------
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
BASE_URL = "http://127.0.0.1:8000/api/1.0"

EMAIL = "test@example.com"
PASSWORD = "password123"

# Portfolio UUID
PORTFOLIO_ID = "591dd7f5-fff1-4041-ae74-3507688da719"

# --------------------
# 1. Get Supabase Access Token
# --------------------
auth_res = requests.post(
    f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
    headers={
        "apikey": SUPABASE_KEY,
        "Content-Type": "application/json"
    },
    json={
        "email": EMAIL,
        "password": PASSWORD
    }
)

if auth_res.status_code != 200:
    print("❌ Failed to log in:", auth_res.status_code, auth_res.text)
    exit(1)

access_token = auth_res.json()["access_token"]
print(f"✅ Logged in as {EMAIL}")
print(f"Access Token: {access_token[:20]}...")

headers = {
    "Authorization": f"Bearer {access_token}"
}

res = requests.get(f"{BASE_URL}/portfolios/{PORTFOLIO_ID}/orders", headers=headers)

print("\nStatus:", res.status_code)
try:
    print(json.dumps(res.json(), indent=4))
except Exception:
    print(res.text)


✅ Logged in as test@example.com
Access Token: eyJhbGciOiJIUzI1NiIs...

Status: 404
{
    "detail": "Not Found"
}
